In [1]:
import Orange
import glob
import pickle
import h5py
import numpy as np
import pandas as pd
import dask.array as da

from Orange.data import Domain, ContinuousVariable, StringVariable

TCGA_FILENAMES = glob.glob('/Users/jakakokosar/OneDrive/dev/data/TCGA/TCGA-*[!metas].csv')
df = pd.concat(map(pd.read_csv, TCGA_FILENAMES))
df = df.set_index('sample_id')


In [19]:
atts = [ContinuousVariable(gene) for gene in df.columns] 
domain = Domain(atts, None, [StringVariable(df.index.name)])

with h5py.File('tcga-samples.hdf5', 'w') as f:
    f.create_dataset('X', data=df.values)
    f.create_dataset('domain', data=np.void(pickle.dumps(domain)))
    f.create_dataset('metas', data=np.void(pickle.dumps(np.asarray(df.index).reshape(-1, 1))))

In [25]:
with h5py.File('tcga-samples.hdf5', 'r') as f:
    domain = pickle.loads(f['domain'][()])
    metas = pickle.loads(f['metas'][()])
    X = da.from_array(f['X'])
    print(X, metas.shape, len(domain))

dask.array<array, shape=(11093, 60483), dtype=float64, chunksize=(4096, 4096), chunktype=numpy.ndarray> (11093, 1) 60484
